In [32]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

page = requests.get(" https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
page.status_code
page.content

soup = BeautifulSoup(page.content, 'lxml')
  
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
#print(df[0].to_json(orient='records'))
df = df[0]
df.head(12)


,0,1,2
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned


In [33]:
##cleaning data - removing Not assigned boroughs and replacing not assigned neighbourhoods with boroughs.
df.loc[0,:]
df.columns = df.loc[0,:]#['a', 'b']
df.drop([0],axis=0,inplace = True)
df = df[df['Borough']!='Not assigned']  # removing entries with no value for Borough
df = df.reset_index()                   #resetting index after removing
df.drop(['index'],axis=1,inplace = True)
temp = df[df['Neighbourhood']=='Not assigned'].index.tolist() # getting a list of Neighbourhoods with the value "Not assigned."
df.loc[temp,'Neighbourhood'] = df.loc[temp,'Borough'] ## replacing those neighbourhoods with borough name
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [34]:
#finding and grouping Neighborhoods with the same Post code
for index, row in df.iterrows():
      row['Neighbourhood'] =  df[df['Postcode']== row['Postcode']]['Neighbourhood'].str.cat(sep=',')

        
df.drop_duplicates(subset='Postcode', keep='first', inplace=True) #dropping the duplicates
df.head(12)


,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
4,M6A,North York,"Lawrence Heights,Lawrence Manor"
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,"Rouge,Malvern"
10,M3B,North York,Don Mills North
11,M4B,East York,"Woodbine Gardens,Parkview Hill"
13,M5B,Downtown Toronto,"Ryerson,Garden District"


In [38]:
df.shape

(103, 3)